# Construct Adjoint for Julia v1

* Date created: June 12, 2019
* Name: Sultan Aitzhan
* Description: The relevant code for constructing an adjoint matrix is copypasted here from $\textit{Fokas_method_code_v1.ipynb}$ and the test functions are updated.

## Importing packages and modules

In [12]:
using SymPy
# using Roots
using Distributions
# using IntervalArithmetic
# using IntervalRootFinding
using ApproxFun

## Global variables

In [13]:
TOL = 1e-05
DIGITS = 3
INFTY = 10

10

## Helper functions

### `check_all`

In [14]:
# Check whether all elements in a not necessarily homogeneous array satisfy a given condition.
function check_all(array, condition)
    for x in array
        if !condition(x)
            return false
        end
    end
    return true
end

check_all (generic function with 1 method)

### `set_tol`

In [15]:
function set_tol(x::Union{Number, Array}, y::Union{Number, Array}; atol = TOL)
    if isa(x, Number) && isa(y, Number)
       return atol * mean([x y])
    elseif isa(x, Array) && isa(y, Array)
        if size(x) != size(y)
            throw(error("Array dimensions do not match"))
        end
        # Avoid InexactError() when taking norm()
        x = convert(Array{Complex}, x)
        y = convert(Array{Complex}, y)
        return atol * (LinearAlgebra.norm(x) + LinearAlgebra.norm(y))
    else
        throw(error("Invalid input"))
    end
end

set_tol (generic function with 1 method)

### `evaluate`

In [16]:
function evaluate(func::Union{Function, SymPy.Sym, Number}, a::Number)
    if isa(func, Function)
        funcA = func(a)
    elseif isa(func, SymPy.Sym) # SymPy.Sym must come before Number because SymPy.Sym will be recognized as Number
        freeSymbols = free_symbols(func)
        if length(freeSymbols) > 1
            throw(error("func should be univariate"))
        elseif length(freeSymbols) == 1
            t = free_symbols(func)[1,1]
            if isa(a, SymPy.Sym) # if x is SymPy.Sym, do not convert result to Number to preserve pretty printing
                funcA = subs(func, t, a)
            else
                funcA = SymPy.N(subs(func, t, a))
            end
        else
            funcA = func
        end
    else # func is Number
        funcA = func
    end
    return funcA
end

evaluate (generic function with 1 method)

### `partition`

In [17]:
# Generate two-integer partitions of n
function partition(n::Int)
    output = []
    for i = 0:n
        j = n - i
        push!(output, (i,j))
    end
    return output
end

partition (generic function with 1 method)

### `symDeriv`

In [18]:
function get_deriv(u::Union{SymPy.Sym, Number}, k::Int)
    if k < 0
        throw(error("Non-negative k required"))
    end
    if isa(u, SymPy.Sym)
        freeSymbols = free_symbols(u)
        if length(freeSymbols) > 1
            throw(error("u should be univariate"))
        elseif length(freeSymbols) == 1
            t = freeSymbols[1]
            y = u
            for i = 1:k
                newY = diff(y, t) # diff is from SymPy
                y = newY
            end
            return y
        else
            if k > 0
                return 0
            else
                return u
            end
        end
    else
        if k > 0
            return 0
        else
            return u
        end
    end
end

get_deriv (generic function with 1 method)

### `add_func`

In [19]:
# Function addition (f + g)(x) := f(x) + g(x)
function add_func(f::Union{Number, Function}, g::Union{Number, Function})
    function h(x)
        if isa(f, Number)
            if isa(g, Number)
                return f + g
            else
                return f + g(x)
            end
        elseif isa(f, Function)
            if isa(g, Number)
                return f(x) + g
            else
                return f(x) + g(x)
            end
        end
    end
    return h
end

add_func (generic function with 1 method)

### `mult_func`

In [20]:
# Function multiplication (f * g)(x) := f(x) * g(x)
function mult_func(f::Union{Number, Function}, g::Union{Number, Function})
    function h(x)
        if isa(f, Number)
            if isa(g, Number)
                return f * g
            else
                return f * g(x)
            end
        elseif isa(f, Function)
            if isa(g, Number)
                return f(x) * g
            else
                return f(x) * g(x)
            end
        end
    end
    return h
end

mult_func (generic function with 1 method)

### `evaluate_matrix`

In [10]:
# Evaluate a matrix at t=a.
# Entries of B may be Function, Number, or Sympy.Sym.
function evaluate_matrix(matrix::Array, a::Number, t=nothing)
    (m, n) = size(matrix)
    matrixA = Array{Number}(m,n)
    for i = 1:m
        for j = 1:n
            matrixA[i,j] = evaluate(matrix[i,j], a, t)
        end
    end
    return matrixA
end

evaluate_matrix (generic function with 2 methods)

### `get_polynomial`

In [11]:
# Implement a polynomial in the form of Julia function given an array containing coefficients of x^n, x^{n-1},..., x^2, x, 1.
function get_polynomial(coeffList::Array)
    polynomial = 0
    n = length(coeffList)-1
    for i in 0:n
        newTerm = t -> coeffList[i+1] * t^(n-i)
        polynomial = add_func(polynomial, newTerm)
    end
    return polynomial
end

get_polynomial (generic function with 1 method)

### `get_polynomialDeriv`

In [12]:
# Get the kth derivative of a polynomial implemented above
function get_polynomialDeriv(coeffList::Array, k::Int)
    if k < 0
        throw(error("Only nonnegative degrees are allowed"))
    elseif k == 0
        newCoeffList = coeffList
    else
        for counter = 1:k
            n = length(coeffList)
            newCoeffList = hcat([0],[(n-i)*coeffList[i] for i in 1:(n-1)]')
            coeffList = newCoeffList
        end
    end
    return get_polynomial(newCoeffList)
end

get_polynomialDeriv (generic function with 1 method)

## Structs

### `StructDefinitionError`

In [10]:
struct StructDefinitionError <: Exception
    msg::String
end

### `SymLinearDifferentialOperator`

In [21]:
struct SymLinearDifferentialOperator
    # Entries in the array should be SymPy.Sym or Number. SymPy.Sym seems to be a subtype of Number, i.e., Array{Union{Number,SymPy.Sym}} returns Array{Number}. But specifying symPFunctions as Array{Number,2} gives a MethodError when the entries are Sympy.Sym objects.
    symPFunctions::Array
    interval::Tuple{Number,Number}
    t::SymPy.Sym
    SymLinearDifferentialOperator(symPFunctions::Array, interval::Tuple{Number,Number}, t::SymPy.Sym) =
    try
        symL = new(symPFunctions, interval, t)
        check_symLinearDifferentialOperator_input(symL)
        return symL
    catch err
        throw(err)
    end
end

function check_symLinearDifferentialOperator_input(symL::SymLinearDifferentialOperator)
    symPFunctions, (a,b), t = symL.symPFunctions, symL.interval, symL.t
    for symPFunc in symPFunctions
        if isa(symPFunc, SymPy.Sym)
            if size(free_symbols(symPFunc)) != (1,) && size(free_symbols(symPFunc)) != (0,)
                throw(StructDefinitionError(:"Only one free symbol is allowed in symP_k"))
            end
        elseif !isa(symPFunc, Number)
            throw(StructDefinitionError(:"symP_k should be SymPy.Sym or Number"))
        end
    end
    return true
end

check_symLinearDifferentialOperator_input (generic function with 1 method)

### `LinearDifferentialOperator`

In [15]:
# A linear differential operator of order n is encoded by an 1 x (n+1) array of functions, an interval [a,b], and its symbolic expression.
# symL is an attribute of L that needs to be input by the user. There are checks to make sure symL is indeed the symbolic version of L.
# Principle: Functionalities of Julia Functions >= Functionalities of SymPy. If p_k has no SymPy representation, the only consequence should be that outputs by functions that take L as arugment has no symbolic expression. E.g., we allow L.pFunctions and L.symL.pFunctions to differ.
struct LinearDifferentialOperator
    pFunctions::Array # Array of julia functions or numbers representing constant functions
    interval::Tuple{Number,Number}
    symL::SymLinearDifferentialOperator
    LinearDifferentialOperator(pFunctions::Array, interval::Tuple{Number,Number}, symL::SymLinearDifferentialOperator) =
    try
        L = new(pFunctions, interval, symL)
        check_linearDifferentialOperator_input(L)
        return L
    catch err
        throw(err)
    end
end

# Assume symFunc has only one free symbol, as required by the definition of SymLinearDifferentialOperator. 
# That is, assume the input symFunc comes from SymLinearDifferentialOperator.
function check_func_sym_equal(func::Union{Function,Number}, symFunc, interval::Tuple{Number,Number}, t::SymPy.Sym) # symFunc should be Union{SymPy.Sym, Number}, but somehow SymPy.Sym gets ignored
    (a,b) = interval
    # Randomly sample 1000 points from (a,b) and check if func and symFunc agree on them
    for i = 1:1000
        # Check endpoints
        if i == 1
            x = a
        elseif i == 2
            x = b
        else
            x = rand(Uniform(a,b), 1)[1,1]
        end
        funcEvalX = evaluate(func, x)
        if isa(symFunc, SymPy.Sym)
            symFuncEvalX = SymPy.N(subs(symFunc,t,x))
            # N() converts SymPy.Sym to Number
            # https://docs.sympy.org/latest/modules/evalf.html
            # subs() works no matter symFunc is Number or SymPy.Sym
        else
            symFuncEvalX = symFunc
        end
        tol = set_tol(funcEvalX, symFuncEvalX)
        if !isapprox(real(funcEvalX), real(symFuncEvalX); atol = real(tol)) ||
            !isapprox(imag(funcEvalX), imag(symFuncEvalX); atol = imag(tol))
            println("x = $x")
            println("symFunc = $symFunc")
            println("funcEvalX = $funcEvalX")
            println("symFuncEvalX = $symFuncEvalX")
            return false
        end
    end
    return true
end

# Check whether the inputs of L are valid.
function check_linearDifferentialOperator_input(L::LinearDifferentialOperator)
    pFunctions, (a,b), symL = L.pFunctions, L.interval, L.symL
    symPFunctions, t = symL.symPFunctions, symL.t
    # domainC = Complex(a..b, 0..0) # Domain [a,b] represented in the complex plane
    p0 = pFunctions[1]
    # p0Chebyshev = Fun(p0, a..b) # Chebysev polynomial approximation of p0 on [a,b]
    if !check_all(pFunctions, pFunc -> (isa(pFunc, Function) || isa(pFunc, Number)))
        throw(StructDefinitionError(:"p_k should be Function or Number"))
    elseif length(pFunctions) != length(symPFunctions)
        throw(StructDefinitionError(:"Number of p_k and symP_k do not match"))
    elseif (a,b) != symL.interval
        throw(StructDefinitionError(:"Intervals of L and symL do not match"))
    # # Assume p_k are in C^{n-k}. Check whether p0 vanishes on [a,b]. 
    # # roots() in IntervalRootFinding doesn't work if p0 is sth like t*im - 2*im. Neither does find_zero() in Roots.
    # # ApproxFun.roots() 
    # elseif (isa(p0, Function) && (!isempty(roots(p0Chebyshev)) || all(x->x>b, roots(p0Chebyshev)) || all(x->x<b, roots(p0Chebyshev)) || p0(a) == 0 || p0(b) == 0)) || p0 == 0 
    #     throw(StructDefinitionError(:"p0 vanishes on [a,b]"))
    elseif !all(i -> check_func_sym_equal(pFunctions[i], symPFunctions[i], (a,b), t), 1:length(pFunctions))
        # throw(StructDefinitionError(:"symP_k does not agree with p_k on [a,b]"))
        warn("symP_k does not agree with p_k on [a,b]") # Make this a warning instead of an error because the functionalities of Julia Functions may be more than those of SymPy objects; we do not want to compromise the functionalities of LinearDifferentialOperator because of the restrictions on SymPy.
    else
        return true
    end
end

check_linearDifferentialOperator_input (generic function with 1 method)

### `VectorBoundaryForm`

In [16]:
# A boundary condition Ux = 0 is encoded by an ordered pair of two matrices (M, N) whose entries are Numbers.
struct VectorBoundaryForm
    M::Array # Why can't I specify Array{Number,2} without having a MethodError?
    N::Array
    VectorBoundaryForm(M::Array, N::Array) =
    try
        U = new(M, N)
        check_vectorBoundaryForm_input(U)
        return U
    catch err
        throw(err)
    end
end

# Check whether the input matrices that characterize U are valid
function check_vectorBoundaryForm_input(U::VectorBoundaryForm)
    # M, N = U.M, U.N
    # Avoid Inexact() error when taking rank()
    M = convert(Array{Complex}, U.M)
    N = convert(Array{Complex}, U.N)
    if !(check_all(U.M, x -> isa(x, Number)) && check_all(U.N, x -> isa(x, Number)))
        throw(StructDefinitionError(:"Entries of M, N should be Number"))
    elseif size(U.M) != size(U.N)
        throw(StructDefinitionError(:"M, N dimensions do not match"))
    elseif size(U.M)[1] != size(U.M)[2]
        throw(StructDefinitionError(:"M, N should be square matrices"))
    elseif rank(hcat(M, N)) != size(M)[1] # rank() throws weird "InexactError()" when taking some complex matrices
        throw(StructDefinitionError(:"Boundary operators not linearly independent"))
    else
        return true
    end
end

check_vectorBoundaryForm_input (generic function with 1 method)

## Main functions

### `get_L`

In [17]:
# Construct L from symL by turning symPFunctions to Julia Function objects
function get_L(symL::SymLinearDifferentialOperator)
    symPFunctions, (a,b), t = symL.symPFunctions, symL.interval, symL.t
    pFunctions = Array{Union{Function, Number}}(1,length(symPFunctions))
    for i =1:length(symPFunctions)
        symPFunc = symPFunctions[i]
        if isa(symPFunc, SymPy.Sym) # if symPFunc is a SymPy.Sym object
            if t in free_symbols(symPFunc)
                pFunc = lambdify(symPFunc)
            else
                pFunc = SymPy.N(symPFunc)
            end
        else # symPFunc could be a Number object
            pFunc = symPFunc
        end
        pFunctions[i] = pFunc
    end
    L = LinearDifferentialOperator(pFunctions, (a,b), symL)
    return L
end

get_L (generic function with 1 method)

### `get_URank`

In [18]:
# Calculate the rank of U, i.e., rank(M:N)
function get_URank(U::VectorBoundaryForm)
    # Avoid InexactError() when taking hcat() and rank()
    M = convert(Array{Complex}, U.M)
    N = convert(Array{Complex}, U.N)
    MHcatN = hcat(M, N)
    return rank(MHcatN)
end

get_URank (generic function with 1 method)

### `get_Uc`

In [19]:
# Find Uc, a complementary form of U
function get_Uc(U::VectorBoundaryForm)
    try
        check_vectorBoundaryForm_input(U)
        n = get_URank(U)
        I = complex(eye(2*n))
        M, N = U.M, U.N
        MHcatN = hcat(M, N)
        # Avoid InexactError() when taking rank()
        mat = convert(Array{Complex}, MHcatN)
        for i = 1:(2*n)
            newMat = vcat(mat, I[i:i,:])
            newMat = convert(Array{Complex}, newMat)
            if rank(newMat) == rank(mat) + 1
                mat = newMat
            end
        end
        UcHcat = mat[(n+1):(2n),:]
        Uc = VectorBoundaryForm(UcHcat[:,1:n], UcHcat[:,(n+1):(2n)])
        return Uc
    catch err
        return err
    end
end

get_Uc (generic function with 1 method)

### `get_H`

In [20]:
# Construct H from M, N, Mc, Nc
function get_H(U::VectorBoundaryForm, Uc::VectorBoundaryForm)
    MHcatN = hcat(convert(Array{Complex}, U.M), convert(Array{Complex}, U.N))
    McHcatNc = hcat(convert(Array{Complex}, Uc.M), convert(Array{Complex}, Uc.N))
    H = vcat(MHcatN, McHcatNc)
    return H
end

get_H (generic function with 1 method)

### `get_pStringMatrix`

In [21]:
# Construct a matrix whose ij-entry is a string "pij" which denotes the (j-1)th derivative of p_(i-1)
function get_pStringMatrix(L::LinearDifferentialOperator)
    if isa(L, LinearDifferentialOperator)
        pFunctions = L.pFunctions
    else
        pFunctions = L.symPFunctions
    end
    n = length(pFunctions)-1
    pStringMatrix = Array{String}(n,n)
    for i in 0:(n-1)
        for j in 0:(n-1)
            pStringMatrix[i+1,j+1] = string("p", i,j)
        end
    end
    return pStringMatrix
end

get_pStringMatrix (generic function with 1 method)

### `get_symPDerivMatrix`

In [22]:
# Construct a matrix whose ij-entry is the symbolic expression of the (j-1)th derivative of p_(i-1).
# Functions with keyword arguments are defined using a semicolon in the signature.

function get_symPDerivMatrix(L::LinearDifferentialOperator; substitute = true)
    symL = L.symL
    symPFunctions, t = symL.symPFunctions, symL.t
    n = length(symPFunctions)-1
    symPDerivMatrix = Array{SymPy.Sym}(n,n)
    if substitute
        pFunctionSymbols = symPFunctions
    else
        pFunctionSymbols = [SymFunction(string("p", i))(t) for i in 0:(n-1)]
    end
    for i in 1:n
        for j in 1:n
            index, degree = i-1, j-1
            symP = pFunctionSymbols[index+1]
            if !isa(symP, SymPy.Sym)
                if degree > 0
                    symPDeriv = 0
                else
                    symPDeriv = symP
                end
            else
                symPDeriv = symDeriv(symP, t, degree)
            end
            symPDerivMatrix[i,j] = symPDeriv
        end
    end
    return symPDerivMatrix
end

# function get_symPDerivMatrix(L::LinearDifferentialOperator; substitute = true)
#     symL = L.symL
#     symPFunctions, t = symL.symPFunctions, symL.t
#     n = length(symPFunctions)-1
#     symPDerivMatrix = Array{SymPy.Sym}(n,n)
#     if substitute
#         pFunctionSymbols = symPFunctions
#     else
#         pFunctionSymbols = [SymFunction(string("p", i))(t) for i in 0:(n-1)]
#     end
#     for i in 1:n
#         for j in 1:n
#             index, degree = i-1, j-1
#             symPDeriv = pFunctionSymbols[index+1]
#             symPDerivMatrix[i,j] = symDeriv(symPDeriv, t, degree)
#         end
#     end
#     return symPDerivMatrix
# end

# For L, the above matrix would need to be constructed by hand.

get_symPDerivMatrix (generic function with 1 method)

### `get_symUvForm`

In [23]:
# Create the symbolic expression for [uv](t).
# If substitute is true: Substitute the p_k SymFunctions with SymPy.Sym definitions, e.g., substitute p0 by t + 1.
function get_symUvForm(L::LinearDifferentialOperator, u::SymPy.Sym, v::SymPy.Sym; substitute = true)
    symL = L.symL
    symPFunctions, t = symL.symPFunctions, symL.t
    n = length(symPFunctions)-1
    if substitute
        pFunctionSymbols = symPFunctions
    else
        pFunctionSymbols = [SymFunction(string("p", i))(t) for i in 0:(n-1)]
    end
    sum = 0
    for m = 1:n
        for (j,k) in partition(m-1)
            summand = (-1)^j * symDeriv(u, t, k) * symDeriv(pFunctionSymbols[n-m+1] * conj(v), t, j)
            sum += summand
        end
    end
    sum = expand(sum)
    return sum
end

get_symUvForm (generic function with 1 method)

### `get_Bjk`

In [24]:
# Find Bjk using explicit formula
function get_Bjk(L::LinearDifferentialOperator, j::Int, k::Int; symbolic = false, substitute = true, pDerivMatrix::Array = [])
    n = length(L.pFunctions)-1
    sum = 0
    if symbolic
        symPDerivMatrix = get_symPDerivMatrix(L; substitute = substitute)
        for l = (j-1):(n-k)
            summand = binomial(l, j-1) * symPDerivMatrix[n-k-l+1, l-j+1+1] * (-1)^l
            sum += summand
        end
    else
        if isempty(pDerivMatrix)
            throw(error("pDerivMatrix required"))
        elseif size(pDerivMatrix) != (n,n)
            throw(error("Size of pDerivMatrix should be ($n,$n)"))
        end
        for l = (j-1):(n-k)
            summand = mult_func(binomial(l, j-1) * (-1)^l, pDerivMatrix[n-k-l+1, l-j+1+1])
            sum = add_func(sum, summand)
        end
    end
    return sum
end

get_Bjk (generic function with 1 method)

### `get_B`

In [25]:
# Construct the B matrix using explicit formula
function get_B(L::LinearDifferentialOperator; symbolic = false, substitute = true, pDerivMatrix::Array = [])
    n = length(L.pFunctions)-1
    B = Array{Union{Function, Number}}(n,n)
    for j = 1:n
        for k = 1:n
            B[j,k] = get_Bjk(L, j, k; symbolic = symbolic, substitute = substitute, pDerivMatrix = pDerivMatrix)
        end
    end
    return B
end

get_B (generic function with 1 method)

### `get_BHat`

In [26]:
# Construct B_hat. Since all entries of B_hat are evaluated, BHat is a numeric matrix.
function get_BHat(L::LinearDifferentialOperator, B::Array)
    pFunctions, (a,b) = L.pFunctions, L.interval
    n = length(pFunctions)-1
    BHat = Array{Complex}(2n,2n)
    BEvalA = evaluate_matrix(B, a)
    BEvalB = evaluate_matrix(B, b)
    BHat[1:n,1:n] = -BEvalA
    BHat[(n+1):(2n),(n+1):(2n)] = BEvalB
    BHat[1:n, (n+1):(2n)] = 0
    BHat[(n+1):(2n), 1:n] = 0
    return BHat
end

get_BHat (generic function with 1 method)

### `get_J`

In [27]:
# Construct J = (B_hat * H^{(-1)})^*, where ^* denotes conjugate transpose
function get_J(BHat, H)
    n = size(H)[1]
    H = convert(Array{Complex}, H)
    J = (BHat * inv(H))'
    # J = convert(Array{Complex}, J)
    return J
end

get_J (generic function with 1 method)

### `get_adjoint`

In [28]:
# Construct U+
function get_adjoint(J)
    n = convert(Int, size(J)[1]/2)
    J = convert(Array{Complex}, J)
    PStar = J[(n+1):2n,1:n]
    QStar = J[(n+1):2n, (n+1):2n]
    adjointU = VectorBoundaryForm(PStar, QStar)
    return adjointU
end

get_adjoint (generic function with 1 method)

### `get_symXi`

In [29]:
# Construct the symbolic expression of \xi = [x; x'; x''; ...], an n x 1 vector of derivatives of x(t)
function get_symXi(L::LinearDifferentialOperator; substitute = false, xDef = nothing)
    n = length(L.pFunctions)-1
    t = symbols("t")
    symXi = Array{SymPy.Sym}(n,1)
    if !substitute
        xDef = SymFunction("x")(t)
    end
    for i = 1:n
        try
            symXi[i] = symDeriv(xDef,t,i-1)
        catch err
            if isa(err, MethodError)
                throw(error("Definition of x required"))
            end
        end
    end
    return symXi
end

# For L, \xi needs to be contructed by hand, e.g., xi = [t->t^2+1; t->2t; t->2; ...]

get_symXi (generic function with 1 method)

### `evaluate_xi`

In [30]:
# Evaluate \xi at a.
# \xi could be Function, SymPy.Sym, or Number.
function evaluate_xi(L::LinearDifferentialOperator, a::Number, xi)
    if any(xDeriv->isa(xDeriv, SymPy.Sym), xi)
        t = L.symL.t
    else
        t = nothing
    end
    n = length(xi)
    xiEvalA = Array{Number}(n,1)
    for i = 1:n
        xiEvalA[i,1] = evaluate(xi[i,1],a,t)
    end
    return xiEvalA
end

evaluate_xi (generic function with 1 method)

### `get_boundaryCondition`

In [31]:
# Get boundary condition Ux = M\xi(a) + N\xi(b)
function get_boundaryCondition(L::LinearDifferentialOperator, U::VectorBoundaryForm, xi)
    # xi cannot contain other types than Function or Number
    if !all(xDeriv->isa(xDeriv, Union{Function, Number}), xi)
        typeXi = typeof(xi)
        throw(error("xi of type $typeXi does not match L of type LinearDifferentialOperator"))
    end
    (a,b) = L.interval
    M, N = U.M, U.N
    xiEvalA = evaluate_xi(L, a, xi)
    xiEvalB = evaluate_xi(L, b, xi)
    Ux = M*xiEvalA + N*xiEvalB
    return Ux
end

get_boundaryCondition (generic function with 1 method)

### `check_adjoint`

In [32]:
# Check if U+ is valid (only works for homogeneous cases Ux=0)
function check_adjoint(L::LinearDifferentialOperator, U::VectorBoundaryForm, adjointU::VectorBoundaryForm, B::Array)
    (a,b) = L.interval
    M, N = U.M, U.N
    P, Q = (adjointU.M)', (adjointU.N)'
    # Avoid InexactError() when taking inv()
    BEvalA = convert(Array{Complex}, evaluate_matrix(B, a))
    BEvalB = convert(Array{Complex}, evaluate_matrix(B, b))
    left = M * inv(BEvalA) * P
    right = N * inv(BEvalB) * Q
    # println("M * inv(BEvalA) * P = $left")
    # println("N * inv(BEvalB) * Q = $right")
    tol = set_tol_matrix(left, right)
    return all(i -> isapprox(left[i], right[i]; atol = tol), length(left)) # Can't use == to deterimine equality because left and right are arrays of floats
end

check_adjoint (generic function with 1 method)

### `construct_validAdjoint`

In [33]:
# Find a valid adjoint
function construct_validAdjoint(L::LinearDifferentialOperator, U::VectorBoundaryForm, pDerivMatrix::Array)
    B = get_B(L; pDerivMatrix = pDerivMatrix)
    BHat = get_BHat(L, B)
    Uc = get_Uc(U)
    H = get_H(U, Uc)
    J = get_J(BHat, H)
    adjointU = get_adjoint(J)
    if check_adjoint(L, U, adjointU, B)
        return adjointU
    else
        throw(error("Adjoint found not valid"))
    end
end

construct_validAdjoint (generic function with 1 method)

## Testing

### Helper functions

#### `generate_symPFunctions`

In [1]:
function generate_symPFunctions(n; random = false, constant = false)
    global t = symbols("t")
    if random
        symPFunctions = Array{Number}(undef, 1,n+1)
        for i = 1:(n+1)
            seed = rand(0:1)
            if seed == 0 # constant
                symPFunctionRe = rand(Uniform(1.0,10.0), 1, 1)[1]
                symPFunctionIm = rand(Uniform(1.0,10.0), 1, 1)[1]
                symPFunction = symPFunctionRe + symPFunctionIm*im
            else # variable
                coeffsNo = rand(1:5)
                pFunctionCoeffsRe = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffsIm = rand(Uniform(1.0,10.0), 1,  coeffsNo)
                pFunctionCoeffs = pFunctionCoeffsRe + pFunctionCoeffsIm*im
                symPFunction = sum([pFunctionCoeffs[i+1]*t^(length(pFunctionCoeffs)-1-i) for i in 0:(length(pFunctionCoeffs)-1)])
            end
            symPFunctions[i] = symPFunction
        end
    else
        if constant # constant
            symPFunctionsRe = rand(Uniform(1.0,10.0), 1, (n+1))
            symPFunctionsIm = rand(Uniform(1.0,10.0), 1, (n+1))
            symPFunctions = symPFunctionsRe + symPFunctionsIm*im
        else # variable
            symPFunctions = Array{Number}(undef, 1,n+1)
            for i = 1:(n+1)
                # Each p_k is a polynomial function with random degree between 0 to 4 and random coefficients between 0 and 10
                coeffsNo = rand(1:5)
                pFunctionCoeffsRe = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffsIm = rand(Uniform(1.0,10.0), 1,  coeffsNo)
                pFunctionCoeffs = pFunctionCoeffsRe + pFunctionCoeffsIm*im
                symPFunction = sum([pFunctionCoeffs[i+1]*t^(length(pFunctionCoeffs)-1-i) for i in 0:(length(pFunctionCoeffs)-1)])
                symPFunctions[i] = symPFunction
            end
        end
    end
    return symPFunctions
end 

generate_symPFunctions (generic function with 1 method)

#### `generate_pFunctions`

In [2]:
function generate_pFunctions(n; random = false, constant = false)
    if random
        pFunctions = Array{Union{Function, Number}}(undef,1,n+1)
        pDerivMatrix = Array{Union{Function, Number}}(undef, n,n)
        for i = 1:(n+1)
            seed = rand(0:1)
            if seed == 0 # constant
                pFunctionRe = rand(Uniform(1.0,10.0), 1, 1)[1]
                pFunctionIm = rand(Uniform(1.0,10.0), 1, 1)[1]
                pFunction = pFunctionRe + pFunctionIm*im
                if i < n+1
                    pDerivMatrix[i,1] = pFunction
                    pDerivMatrix[i:i, 2:n] = zeros(1, n-1)
                end
            else # variable
                coeffsNo = rand(1:5)
                pFunctionCoeffsRe = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffsIm = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffs = pFunctionCoeffsRe + pFunctionCoeffsIm*im
                pFunction = get_polynomial(pFunctionCoeffs)
                if i < n+1
                    pDerivMatrix[i:i, 1:n] = [get_polynomialDeriv(pFunctionCoeffs, k) for k = 0:(n-1)]
                end
            end
            pFunctions[i] = pFunction
        end
    else
        if constant # constant
            pFunctionsRe = rand(Uniform(1.0,10.0), 1, (n+1))
            pFunctionsIm = rand(Uniform(1.0,10.0), 1, (n+1))
            pFunctions = pFunctionsRe + pFunctionsIm*im
            pDerivMatrix = complex(Matrix{Float64}(LinearAlgebra.I, n, n))
            for i = 1:n
                for j = 1:n
                    if j == 1
                        pDerivMatrix[i,j] = pFunctions[i]
                    else
                        pDerivMatrix[i,j] = 0
                    end
                end
            end
        else # variable
            pFunctions = Array{Union{Function, Number}}(undef,1,n+1)
            pDerivMatrix = Array{Union{Function, Number}}(undef,n,n)
            for i = 1:(n+1)
                # Each p_k is a polynomial function with random degree between 0 to 4 and random coefficients between 0 and 10
                coeffsNo = rand(1:5)
                pFunctionCoeffsRe = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffsIm = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffs = pFunctionCoeffsRe + pFunctionCoeffsIm*im
                if i < n+1
                    pDerivMatrix[i:i, 1:n] = [get_polynomialDeriv(pFunctionCoeffs, k) for k = 0:(n-1)]
                end
                pFunction = get_polynomial(pFunctionCoeffs)
                pFunctions[i] = pFunction
            end
        end
    end
    return pFunctions, pDerivMatrix
end

generate_pFunctions (generic function with 1 method)

#### `generate_pFunctionsAndSymPFunctions`

In [3]:
function generate_pFunctionsAndSymPFunctions(n; random = false, constant = false)
    global t = symbols("t")
    if random
        pFunctions = Array{Union{Function, Number}}(undef,1,n+1)
        symPFunctions = Array{Number}(undef, 1,n+1)
        pDerivMatrix = Array{Union{Function, Number}}(undef,n,n)
        for i = 1:(n+1)
            seed = rand(0:1)
            if seed == 0 # constant
                pFunctionRe = rand(Uniform(1.0,10.0), 1, 1)[1]
                pFunctionIm = rand(Uniform(1.0,10.0), 1, 1)[1]
                pFunction = pFunctionRe + pFunctionIm*im
                symPFunction = pFunction
                if i < n+1
                    pDerivMatrix[i,1] = pFunction
                    pDerivMatrix[i:i, 2:n] = zeros(1, n-1)
                end
            else # variable
                coeffsNo = rand(1:5)
                pFunctionCoeffsRe = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffsIm = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffs = pFunctionCoeffsRe + pFunctionCoeffsIm*im
                if i < n+1
                    pDerivMatrix[i:i, 1:n] = [get_polynomialDeriv(pFunctionCoeffs, k) for k = 0:(n-1)]
                end
                pFunction = get_polynomial(pFunctionCoeffs)
                symPFunction = sum([pFunctionCoeffs[i+1]*t^(length(pFunctionCoeffs)-1-i) for i in 0:(length(pFunctionCoeffs)-1)])
            end
            pFunctions[i] = pFunction
            symPFunctions[i] = symPFunction
        end
    else
        if constant # constant
            pFunctionsRe = rand(Uniform(1.0,10.0), 1, (n+1))
            pFunctionsIm = rand(Uniform(1.0,10.0), 1, (n+1))
            pFunctions = pFunctionsRe + pFunctionsIm*im
            symPFunctions = pFunctions
            pDerivMatrix = Array{Union{Function, Number}}(undef,n,n)
            for i = 1:n
                for j = 1:n
                    if j == 1
                        pDerivMatrix[i,j] = pFunctions[i]
                    else
                        pDerivMatrix[i,j] = 0
                    end
                end
            end
        else # variable
            t = symbols("t")
            pFunctions = Array{Function}(undef,1,n+1)
            symPFunctions = Array{Number}(undef, 1,n+1)
            pDerivMatrix = Array{Union{Function, Number}}(undef, n,n)
            for i = 1:(n+1)
                # Each p_k is a polynomial function with random degree between 0 to 4 and random coefficients between 0 and 10
                coeffsNo = rand(1:5)
                pFunctionCoeffsRe = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffsIm = rand(Uniform(1.0,10.0), 1, coeffsNo)
                pFunctionCoeffs = pFunctionCoeffsRe + pFunctionCoeffsIm*im
                if i < n+1
                    pDerivMatrix[i:i, 1:n] = [get_polynomialDeriv(pFunctionCoeffs, k) for k = 0:(n-1)]
                end
                pFunction = get_polynomial(pFunctionCoeffs)
                pFunctions[i] = pFunction
                symPFunction = sum([pFunctionCoeffs[i+1]*t^(length(pFunctionCoeffs)-1-i) for i in 0:(length(pFunctionCoeffs)-1)])
                symPFunctions[i] = symPFunction
            end
        end
    end
    
    return pFunctions, symPFunctions, pDerivMatrix
end

generate_pFunctionsAndSymPFunctions (generic function with 1 method)

### Tests

#### `test_generate_adjoint`

In [4]:
# Test the algorithm to generate valid adjoint U+
function test_generate_adjoint(n, k)
    global results = [true]
    global t = symbols("t")
    global (a,b) = (0,1)

    for counter = 1:k
        println("Test $counter")
        println("Testing the algorithm to generate valid adjoint U+: Constant p_k")
        (pFunctions, symPFunctions, pDerivMatrix) = generate_pFunctionsAndSymPFunctions(n; random = false, constant = true)
        symL = SymLinearDifferentialOperator(symPFunctions, (a,b), t)
        L = LinearDifferentialOperator(pFunctions, (a,b), symL)
        MCandRe = rand(Uniform(1.0,10.0), n, n)
        MCandIm = rand(Uniform(1.0,10.0), n, n)
        MCand = MCandRe + MCandIm*im
        NCandRe = rand(Uniform(1.0,10.0), n, n)
        NCandIm = rand(Uniform(1.0,10.0), n, n)
        NCand = NCandRe + NCandIm*im
        U = VectorBoundaryForm(MCand, NCand)
        println("Testing: order of L = $n")
        passed = false
        try
            adjoint = construct_validAdjoint(L, U, pDerivMatrix)
            passed = true
            append!(results, passed)
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end

        println("Testing the algorithm to generate valid adjoint U+: Variable p_k")
        # Generate variable p_k
        (pFunctions, symPFunctions, pDerivMatrix) = generate_pFunctionsAndSymPFunctions(n; random = false, constant = false)
        symL = SymLinearDifferentialOperator(symPFunctions, (a,b), t)
        L = LinearDifferentialOperator(pFunctions, (a,b), symL)
        MCandRe = rand(Uniform(1.0,10.0), n, n)
        MCandIm = rand(Uniform(1.0,10.0), n, n)
        MCand = MCandRe + MCandIm*im
        NCandRe = rand(Uniform(1.0,10.0), n, n)
        NCandIm = rand(Uniform(1.0,10.0), n, n)
        NCand = NCandRe + NCandIm*im
        U = VectorBoundaryForm(MCand, NCand)
        println("Testing: order of L = $n")
        try
            adjoint = construct_validAdjoint(L, U, pDerivMatrix)
            passed = true
            append!(results, passed)
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end

        println("Testing the algorithm to generate valid adjoint U+: Constant or variable p_k")
        # Generate p_k
        (pFunctions, symPFunctions, pDerivMatrix) = generate_pFunctionsAndSymPFunctions(n; random = true)
        symL = SymLinearDifferentialOperator(symPFunctions, (a,b), t)
        L = LinearDifferentialOperator(pFunctions, (a,b), symL)
        MCandRe = rand(Uniform(1.0,10.0), n, n)
        MCandIm = rand(Uniform(1.0,10.0), n, n)
        MCand = MCandRe + MCandIm*im
        NCandRe = rand(Uniform(1.0,10.0), n, n)
        NCandIm = rand(Uniform(1.0,10.0), n, n)
        NCand = NCandRe + NCandIm*im
        U = VectorBoundaryForm(MCand, NCand)
        println("Testing: order of L = $n")
        try
            adjoint = construct_validAdjoint(L, U, pDerivMatrix)
            passed = true
            append!(results, passed)
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end
    end

    return all(results)
end

test_generate_adjoint (generic function with 1 method)

In [5]:
#for n = 1:10
 #    println(test_generate_adjoint(n, 10))
#end

#### `test_symLinearDifferentialOperatorDef`

In [6]:
# Test the SymLinearDifferentialOperator definition
function test_symLinearDifferentialOperatorDef(n, k)
    global results = [true]
    global t = symbols("t")
    global (a,b) = (0,1)

    for counter = 1:k
        println("Test $counter")
        println("Testing definition of SymLinearDifferentialOperator: symP_k are Function")
        symPFunctions = generate_symPFunctions(n; random = false, constant = false)
        passed = false
        try
            SymLinearDifferentialOperator(symPFunctions, (a,b), t)
            passed = true
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end
        append!(results, passed)

        println("Testing definition of SymLinearDifferentialOperator: symP_k are constant")
        symPFunctions = generate_symPFunctions(n; random = false, constant = true)
        passed = false
        try
            SymLinearDifferentialOperator(symPFunctions, (a,b), t)
            passed = true
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end
        append!(results, passed)

        println("Testing definition of SymLinearDifferentialOperator: symP_k are SymPy.Sym and Number")
        symPFunctions = generate_symPFunctions(n; random = true)
        passed = false
        try
            SymLinearDifferentialOperator(symPFunctions, (a,b), t)
            passed = true
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end
        append!(results, passed)

        println("Testing StructDefinitionError: symP_k should be SymPy.Sym or Number")
        symPFunctions = hcat(generate_symPFunctions(n-1; random = true), ["str"])
        passed = false
        try
            SymLinearDifferentialOperator(symPFunctions, (a,b), t)
        catch err
            if isa(err,StructDefinitionError) && err.msg == "symP_k should be SymPy.Sym or Number"
                passed = true
                println("Passed!")
            else
                println("Failed with $err")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)

        println("Testing StructDefinitionError: Only one free symbol is allowed in symP_k")
        r = symbols("r")
        passed = false
        try
            SymLinearDifferentialOperator([t+1 t+1 r*t+1], (a,b), t)
        catch err
            if isa(err,StructDefinitionError) && err.msg == "Only one free symbol is allowed in symP_k"
                passed = true
                println("Passed!")
            else
                println("Failed with $err")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)
    end

    return all(results)
end

test_symLinearDifferentialOperatorDef (generic function with 1 method)

In [ ]:
for n = 1:10
    println(test_symLinearDifferentialOperatorDef(n, 10))
end

#### `test_linearDifferentialOperatorDef`

In [9]:
# Test the LinearDifferentialOperator definition
function test_linearDifferentialOperatorDef(n, k)
    global results = [true]
    global t = symbols("t")
    global (a,b) = (0,1)
    
    for counter = 1:k
        println("Test $counter")

        # Variable p_k
        println("Testing definition of LinearDifferentialOperator: p_k are Function")
        (pFunctions, symPFunctions, pDerivMatrix) = generate_pFunctionsAndSymPFunctions(n; random = false, constant = false)
        symL = SymLinearDifferentialOperator(symPFunctions, (a,b), t)
        passed = false
        try
            L = LinearDifferentialOperator(pFunctions, (a,b), symL)
            passed = true
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end
        append!(results, passed)

        # Constant coefficients
        println("Testing definition of LinearDifferentialOperator: p_k are Constants")
        (pFunctions, symPFunctions, pDerivMatrix) = generate_pFunctionsAndSymPFunctions(n; random = false, constant = true)
        symL = SymLinearDifferentialOperator(symPFunctions, (a,b), t)
        passed = false
        try
            LinearDifferentialOperator(pFunctions, (a,b), symL)
            passed = true
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end
        append!(results, passed)

        # Mixed coefficients
        println("Testing definition of LinearDifferentialOperator: p_k are mixed")
        # (pFunctions, symPFunctions, pDerivMatrix) = generate_pFunctionsAndSymPFunctions(n; random = true)
        symL = SymLinearDifferentialOperator([1 1 t+1], (a,b), t)
        passed = false
        try
            LinearDifferentialOperator([1 t->1 t->t+1], (a,b), symL)
            passed = true
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end
        append!(results, passed)

        println("Testing StructDefinitionError: p_k should be Function or Number")
        pFunctions = hcat(generate_pFunctions(n-1; random = true)[1], ["str"])
        passed = false
        try
            LinearDifferentialOperator(['s' 1 1], (a,b), symL)
        catch err
            if err.msg == "p_k should be Function or Number" && (isa(err,StructDefinitionError))
                passed = true
                println("Passed!")
            else
                println("Failed with $err")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)

        println("Testing StructDefinitionError: Number of p_k and symP_k do not match")
        symL = SymLinearDifferentialOperator([1 1 t+1], (a,b), t)
        passed = false
        try
            LinearDifferentialOperator([1 t->1], (a,b), symL)
        catch err
            if err.msg == "Number of p_k and symP_k do not match" && (isa(err, StructDefinitionError))
                passed = true
                println("Passed!")
            else
                println("Failed with $err")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)

        println("Testing StructDefinitionError: Intervals of L and symL do not match")
        symL = SymLinearDifferentialOperator([1 1 t+1], (a,b), t)
        passed = false
        try
            LinearDifferentialOperator([1 t->1 t->t+1], (-b,a), symL)
        catch err
            if err.msg == "Intervals of L and symL do not match" && (isa(err, StructDefinitionError))
                passed = true
                println("Passed!")
            else
                println("Failed with $err")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)
    
    end
    return all(results)
end

test_linearDifferentialOperatorDef (generic function with 1 method)

#### `test_vectorBoundaryFormDef`

In [8]:
# Test the VectorBoundaryForm definition
function test_vectorBoundaryFormDef(n, k)
    global results = [true]

    for counter = 1:k
        println("Test $counter")

        println("Testing the definition of VectorBoundaryForm")
        MRe = rand(Uniform(1.0,10.0), n, n)
        MIm = rand(Uniform(1.0,10.0), n, n)
        M = MRe + MIm*im
        NRe = rand(Uniform(1.0,10.0), n, n)
        NIm = rand(Uniform(1.0,10.0), n, n)
        N = NRe + NIm*im
        passed = false
        try
            VectorBoundaryForm(M, N)
            passed = true
        catch err
            println("Failed with $err")
        end
        if passed
            println("Passed!")
        end
        append!(results, passed)

        println("Testing StructDefinitionError: Entries of M, N should be Number")
        M = ['a' 2; 3 4]
        N = M
        passed = false
        try
            VectorBoundaryForm(M, N)
        catch err
            if err.msg == "Entries of M, N should be Number" && isa(err, StructDefinitionError)
                passed = true
                println("Passed!")
            else
                println("Failed with $err")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)

        println("Testing StructDefinitionError: M, N dimensions do not match")
        M = rand(Uniform(1.0,10.0), n, n-1)
        N = rand(Uniform(1.0,10.0), n, n)
        passed = false
        try
            VectorBoundaryForm(M, N)
        catch err
            if err.msg == "M, N dimensions do not match" && isa(err,StructDefinitionError)
                passed = true
                println("Passed!")
            else
                println("Failed with $err")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)

        println("Testing StructDefinitionError: M, N should be square matrices")
        M = rand(Uniform(1.0,10.0), n, n-1)
        N = rand(Uniform(1.0,10.0), n, n-1)
        passed = false
        try
            VectorBoundaryForm(M, N)
        catch err
            if err.msg == "M, N should be square matrices" && isa(err,StructDefinitionError)
                passed = true
                println("Passed!")
            else
                println("Failed with $err")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)

        println("Testing StructDefinitionError: Boundary operators not linearly independent")
        M = [1 2*im; 2 4*im]
        N = [3 4*im; 6 8*im]
        passed = false
        try
            VectorBoundaryForm(M, N)
        catch err
            if err.msg == "Boundary operators not linearly independent" && isa(err,StructDefinitionError)
                passed = true
                println("Passed!")
            end
        end
        if !passed
            println("Failed!")
        end
        append!(results, passed)
    end

    return all(results)
end

test_vectorBoundaryFormDef (generic function with 1 method)